## Second Stage Optimazation of AlphabetSoupCharity
#### Previously removed two columns suspected of being either redundant or skewing predictions
#### Adjusting bin limits of Application Type Up, and Classification Down

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf

#  Import and read the charity_data.csv.
app_df = pd.read_csv("Resources/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#### Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

In [2]:
app_df = app_df.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS", "STATUS"])
app_df.head(1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1


#### Determine the number of unique values in each column.

In [3]:
app_cat = app_df.dtypes.index.tolist()
app_df[app_cat].nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

#### Look at the APPLICATION_TYPE value counts for binning

In [4]:
type_counts = app_df["APPLICATION_TYPE"].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

#### Choose a cutoff value and create a list of application types to be replaced

In [5]:
Infinity = np.inf
app_type_bins = type_counts.value_counts(bins=[0, 700, Infinity])
app_type_bins

(-0.001, 700.0]    10
(700.0, inf]        7
Name: APPLICATION_TYPE, dtype: int64

In [6]:
app_df["COUNT"] = app_df.groupby(["APPLICATION_TYPE"])["APPLICATION_TYPE"].transform('count')
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,COUNT
0,T10,Independent,C1000,ProductDev,Association,0,5000,1,528
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1,27037
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0,1173
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1,27037
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1,27037


In [7]:
other_df = app_df.loc[app_df["COUNT"] < 700]
saved_df = app_df.loc[app_df["COUNT"] >= 700]  

#### Use the variable name `application_types_to_replace`

In [8]:
application_types_to_replace = other_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace

T10    528
T9     156
T13     66
T12     27
T2      16
T25      3
T14      3
T29      2
T15      2
T17      1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
other_df.loc[other_df["APPLICATION_TYPE"] != "Shirley", "APPLICATION_TYPE"] = "Other"
new_type = other_df["APPLICATION_TYPE"].value_counts()
new_type

Other    804
Name: APPLICATION_TYPE, dtype: int64

In [10]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=["COUNT"])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [11]:
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

#### Choose a cutoff value and create a list of classification types to be replaced

In [13]:
class_type_bins = class_counts.value_counts(bins=[0, 700, Infinity])
class_type_bins

(-0.001, 700.0]    65
(700.0, inf]        6
Name: CLASSIFICATION, dtype: int64

In [14]:
application_df["COUNT"] = application_df.groupby(["CLASSIFICATION"])["CLASSIFICATION"].transform('count')
app_df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,COUNT
0,T10,Independent,C1000,ProductDev,Association,0,5000,1,528
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1,27037
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0,1173


In [15]:
other_df = app_df.loc[application_df["COUNT"] < 700]
saved_df = app_df.loc[application_df["COUNT"] >= 700]

#### Use the variable name `classifications_to_replace`

In [16]:
classifications_to_replace = other_df["CLASSIFICATION"].value_counts()
classifications_to_replace

C1700    287
C4000    194
C5000    116
C1270    114
C2700    104
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 65, dtype: int64

In [17]:
other_df.loc[other_df["CLASSIFICATION"] != "Shirley", "CLASSIFICATION"] = "Other"
new_class = other_df["CLASSIFICATION"].value_counts()
new_class

Other    1484
Name: CLASSIFICATION, dtype: int64

In [18]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=["COUNT"])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [19]:
# Check to make sure binning was successful
new_class_counts = application_df['CLASSIFICATION'].value_counts()
new_class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

#### Convert categorical data to numeric with `OneHotEncoder`

In [20]:
# check dataframe types
application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [21]:
# Generate variable list for categories to OnehotEncode
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [22]:
# check list
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [23]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [24]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

In [25]:
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(app_cat)
encode_df.shape

(34299, 48)

In [26]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(app_cat, axis='columns')
application_df.shape

(34299, 50)

In [27]:
application_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Separate Target, Divide train and test sets and Scale

In [28]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=61)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1] # Get the number of columns
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4000      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,461
Trainable params: 6,461
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = model.fit(X_train_scaled,y_train,steps_per_epoch=5,epochs=100)

Epoch 1/100
5/5 [==============================] - 1s 7ms/step - loss: 0.6777 - accuracy: 0.5883
Epoch 2/100
5/5 [==============================] - 0s 8ms/step - loss: 0.6401 - accuracy: 0.6738
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6161 - accuracy: 0.7125
Epoch 4/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5989 - accuracy: 0.7186
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5864 - accuracy: 0.7285
Epoch 6/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5771 - accuracy: 0.7282
Epoch 7/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5703 - accuracy: 0.7271
Epoch 8/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5655 - accuracy: 0.7266
Epoch 9/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5618 - accuracy: 0.7267
Epoch 10/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5590 - accuracy: 0.7267
Epoch 11/100
5/5 [===========

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5584 - accuracy: 0.7262 - 374ms/epoch - 1ms/step
Loss: 0.5584258437156677, Accuracy: 0.7261807322502136


In [34]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity_Optimization2.h5")

#### Slight Decrease in Loss, Slight Increase in Accuracy
#### Still not an improvement, moving onto Neural Network from the original with no adjustments to data processing